In [1]:
import os
os.getcwd()
os.listdir('.')

['.DS_Store',
 '.git',
 '.ipynb_checkpoints',
 'algorithms.txt',
 'extract_feature.py',
 'gender.py',
 'myfirstforest.csv',
 'myfirstforest.py',
 'test.csv',
 'titanic.ipynb',
 'train.csv']

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%pylab inline
import re

Populating the interactive namespace from numpy and matplotlib


In [4]:
data = pd.read_csv('train.csv')
print data.columns
data.dtypes

Index([u'PassengerId', u'Survived', u'Pclass', u'Name', u'Sex', u'Age', u'SibSp', u'Parch', u'Ticket', u'Fare', u'Cabin', u'Embarked'], dtype='object')


PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [6]:
data['Embarked'].isnull().sum()

2

In [7]:
data['Name']

0                               Braund, Mr. Owen Harris
1     Cumings, Mrs. John Bradley (Florence Briggs Th...
2                                Heikkinen, Miss. Laina
3          Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                              Allen, Mr. William Henry
5                                      Moran, Mr. James
6                               McCarthy, Mr. Timothy J
7                        Palsson, Master. Gosta Leonard
8     Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)
9                   Nasser, Mrs. Nicholas (Adele Achem)
10                      Sandstrom, Miss. Marguerite Rut
11                             Bonnell, Miss. Elizabeth
12                       Saundercock, Mr. William Henry
13                          Andersson, Mr. Anders Johan
14                 Vestrom, Miss. Hulda Amanda Adolfina
...
876                    Gustafsson, Mr. Alfred Ossian
877                             Petroff, Mr. Nedelio
878                               Laleff, Mr. Kris

In [8]:
data['Name'].nunique()

891

In [9]:
title = data['Name'].str.findall( (' .{2,6}\.'))
title

0         [ Mr.]
1        [ Mrs.]
2       [ Miss.]
3        [ Mrs.]
4         [ Mr.]
5         [ Mr.]
6         [ Mr.]
7     [ Master.]
8        [ Mrs.]
9        [ Mrs.]
10      [ Miss.]
11      [ Miss.]
12        [ Mr.]
13        [ Mr.]
14      [ Miss.]
...
876      [ Mr.]
877      [ Mr.]
878      [ Mr.]
879     [ Mrs.]
880     [ Mrs.]
881      [ Mr.]
882    [ Miss.]
883      [ Mr.]
884      [ Mr.]
885     [ Mrs.]
886     [ Rev.]
887    [ Miss.]
888    [ Miss.]
889      [ Mr.]
890      [ Mr.]
Name: Name, Length: 891, dtype: object

In [10]:
type(title)

pandas.core.series.Series

In [11]:
print sum(title.apply(len)>1)
#so there is no name parsed into two elements
#print (title.loc[0])
#print type(title.loc[0])
title_str = title.apply(str)
print title_str.nunique()
#title_str.unique()
title_str.value_counts()

0
16


[' Mr.']        517
[' Miss.']      182
[' Mrs.']       125
[' Master.']     40
[' Dr.']          7
[' Rev.']         6
[' Major.']       2
[' Mlle.']        2
[' Col.']         2
[]                2
[' Mme.']         1
[' Don.']         1
[' Ms.']          1
[' Sir.']         1
[' Lady.']        1
[' Capt.']        1
dtype: int64

In [12]:
title_str0 = title_str.apply(lambda x: x[3:-2])
#when use slicing,for the elements that donot have enough length, it will to Empty elements!!
#see below
print title_str0.nunique()
print title_str0.value_counts()
print 'the empty element is: ', title_str0.value_counts().index[9]
#the above will show nothing but PRINT, you cannot tell; if you put it as the last evaluation element, you will see it
#note the differences of outputing here in ipython notebook!
# the last statement will generate the 'Out', others will NOT!!!
title_str0.value_counts().index[9]

16
Mr.        517
Miss.      182
Mrs.       125
Master.     40
Dr.          7
Rev.         6
Mlle.        2
Major.       2
Col.         2
             2
Mme.         1
Ms.          1
Don.         1
Capt.        1
Sir.         1
Lady.        1
dtype: int64
the empty element is:  


''

In [13]:
#replace the target titles
#for all the elements other than the  empty, assign them into the four top categories in the title_str0
# Master is replaced by Mister(Mr.) in 19th century, titanic happned in 1912, so master. meaning boys and young men
# major -> Mr.
# Mlle. -> Miss.
# Ms    -> Mrs.
# Mme. Madame-> Mrs.
# Dr. -> Mr.
# Rev. -> Mr.
# Col. -> Mr.
# Don. -> Mr.
# Capt. -> Mr.
# Sir. -> Mr.
# Lady. -> Mrs.
#check Don.:::
name = data['Name']
title_str0[title_str0 == 'Don.'].index
name.loc[title_str0[title_str0 == 'Don.'].index]

30    Uruchurtu, Don. Manuel E
Name: Name, dtype: object

In [14]:
mr_index = title_str0.apply(lambda x: x in ['Major.','Dr.','Rev.','Col.','Don.','Capt.','Sir.'])
mrs_index = title_str0.apply(lambda x: x in ['Ms.','Mme.','Lady.'])
miss_index = title_str0.apply(lambda x: x in ['Mlle.'])
title_str0[mr_index] = 'Mr.'
title_str0[mrs_index] = 'Mrs.'
title_str0[miss_index] = 'Miss.'
title_str0.value_counts()

Mr.        537
Miss.      184
Mrs.       128
Master.     40
             2
dtype: int64

In [15]:
print name[title_str0[title_str0 ==''].index]
#Countess. -> Mrs.
title_str0.loc[759] = "Mrs."
title_str0.loc[822] = "Mr."
title_str0.value_counts()

759    Rothes, the Countess. of (Lucy Noel Martha Dye...
822                      Reuchlin, Jonkheer. John George
Name: Name, dtype: object


Mr.        538
Miss.      184
Mrs.       129
Master.     40
dtype: int64

In [16]:
data['title'] = title_str0
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,title
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S,Mr.
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C,Mrs.
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss.
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S,Mrs.
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S,Mr.
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,Mr.
6,7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,17463,51.8625,E46,S,Mr.
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,349909,21.0750,NaN,S,Master.
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,11.1333,NaN,S,Mrs.
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,237736,30.0708,NaN,C,Mrs.


In [37]:
#now working on the Ticket, Cabin, Embarked variable
pd.set_option('display.max_rows', 999)
data.loc[:,['Ticket','Cabin', 'Embarked']]
#so ticket number part and string part, we can only retain the number part since MOST of the string parts are missing
# ticket number indicates the seat position, which might be related to the survive likelihood
#one the argument is that for ticket numbers that are similar, if they have some string beforehand, the strings are the same for most of the time
# but, it is definitely not linear, so random forest and etc non-linear models should try this variable extrace from tickets--as float type

#for Cabin, fill all the NaN value as a string, would this tell us something?
#for Embarked, just fill the missing values with the mode, this would be OK

,Ticket,Cabin,Embarked
0,A/5 21171,NaN,S
1,PC 17599,C85,C
2,STON/O2. 3101282,NaN,S
3,113803,C123,S
4,373450,NaN,S
5,330877,NaN,Q
6,17463,E46,S
7,349909,NaN,S
8,347742,NaN,S
9,237736,NaN,C


In [44]:
data.loc[data.Cabin.isnull(), 'Cabin'] = 'unknown'

In [68]:
data.loc[data.Embarked.isnull(),'Embarked'] = data.Embarked.mode().values

In [73]:
data.iloc[[61,829], 11] 

61     S
829    S
Name: Embarked, dtype: object

In [77]:
data['Ticket'].str.findall(' .{1,20}')

0           [ 21171]
1           [ 17599]
2         [ 3101282]
3                 []
4                 []
5                 []
6                 []
7                 []
8                 []
9                 []
10           [ 9549]
11                []
12           [ 2151]
13                []
14                []
15                []
16                []
17                []
18                []
19                []
20                []
21                []
22                []
23                []
24                []
25                []
26                []
27                []
28                []
29                []
30          [ 17601]
31          [ 17569]
32                []
33          [ 24579]
34          [ 17604]
35                []
36                []
37           [ 2152]
38                []
39                []
40                []
41                []
42                []
43           [ 2123]
44                []
45          [ 23567]
46                []
47           